<a href="https://colab.research.google.com/github/ChipMcCallahan/CC1LevelsetProto/blob/main/cc1_levelset_proto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CC1LevelsetProto Demo

#### Install package from GitHub.

In [7]:
%%capture
!pip install git+https://github.com/ChipMcCallahan/CC1LevelsetProto.git

#### Create and name a Levelset.

In [8]:
from cc1_levelset_proto.cc1_levelset_pb2 import Levelset, CC1TileCode

levelset = Levelset()
levelset.name = "My Levelset"
print(levelset)

name: "My Levelset"



#### Add Levels and Populate Basic Fields

In [9]:
for i in range(1,3):
    level = levelset.levels.add()
    level.title = f"Demo Level {i}"
    level.author = "Chip McCallahan"
    level.time = 100 * i
    level.chips = 200 * i
    level.password = ("AAAA", "BBBB")[i-1]
    level.hint = f"This is the hint for level {i}."
print(levelset)

name: "My Levelset"
levels {
  title: "Demo Level 1"
  author: "Chip McCallahan"
  time: 100
  chips: 200
  hint: "This is the hint for level 1."
  password: "AAAA"
}
levels {
  title: "Demo Level 2"
  author: "Chip McCallahan"
  time: 200
  chips: 400
  hint: "This is the hint for level 2."
  password: "BBBB"
}



#### Create a Map

Note that any tilespec that is unspecified defaults to 

`{top: CC1TileCode.FLOOR, bottom: CC1TileCode.FLOOR}`

View the [CC1TileCode enum](https://github.com/ChipMcCallahan/CC1LevelsetProto/blob/main/cc1_levelset_proto/cc1_levelset.proto) for allowed values.

X and Y coordinates must be converted to a single integer coordinate:  
`p = y * 32 + x`. 

These can be unpacked by:  
`x, y = p % 32, p // 32`

In [22]:
TRAPS = {(1, 1): (2, 2), (11, 11): (12, 12)}
CLONERS = {(3, 3): (4, 4), (13, 13): (14, 14)}

# for the demo, just make the same map for each level
for level in levelset.levels:
    for i in range(32):
        for j in range(32):
            tilespec = level.map.tiles[j * 32 + i]

            # add a border of wall tiles
            if i == 0 or j == 0 or i == 31 or j == 31:
                tilespec.top = CC1TileCode.WALL

            # add a second border of teeth
            elif i == 1 or j == 1 or i == 30 or j == 30:
                tilespec.top = CC1TileCode.TEETH_S

                # add each teeth to the movement list
                level.movement.append(j * 32 + i)
            
            # put the player somewhere near the middle
            elif (i, j) == (14, 16):
                tilespec.top = CC1TileCode.PLAYER_S
            
            # and an exit nearby
            elif (i, j) == (20, 20):
                tilespec.top = CC1TileCode.EXIT
            
            # add the traps and cloners
            elif (i, j) in TRAPS.keys():
                tilespec.top = CC1TileCode.TRAP_BUTTON
            elif (i, j) in TRAPS.values():
                tilespec.top = CC1TileCode.TRAP
            elif (i, j) in CLONERS.keys():
                tilespec.top = CC1TileCode.CLONE_BUTTON
            elif (i, j) in CLONERS.values():
                tilespec.bottom = CC1TileCode.CLONER
                tilespec.top = CC1TileCode.FIREBALL_S
            
            # connect the traps and cloners
            for button, trap in TRAPS.items():
                bx, by = button
                tx, ty = trap
                level.trap_controls[by * 32 + bx] = ty * 32 + tx
            for button, cloner in CLONERS.items():
                bx, by = button
                cx, cy = cloner
                level.clone_controls[by * 32 + bx] = cy * 32 + cx


In [24]:
print(f"Clone controls are {levelset.levels[0].clone_controls}")
print(f"Trap controls are {levelset.levels[0].trap_controls}")
print(f"Tile at 14, 16 is: \n{levelset.levels[0].map.tiles[16 * 32 + 14]}")

Clone controls are {429: 462, 99: 132}
Trap controls are {99: 132, 429: 462, 363: 396, 33: 66}
Tile at 15, 15 is: 
top: PLAYER_S



#### Further Exploration
There's not much more to do here, since this is just the proto definition. For further functionality, see the [CC1LevelsetIO repository](https://github.com/ChipMcCallahan/CC1LevelsetIO).